<img src="../logo/logo.jpg">

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Combinar-DataFrames" data-toc-modified-id="Combinar-DataFrames-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Combinar DataFrames</a></div><div class="lev2 toc-item"><a href="#Combinar-tablas-al-estilo-del-álgebra-relacional" data-toc-modified-id="Combinar-tablas-al-estilo-del-álgebra-relacional-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Combinar tablas al estilo del álgebra relacional</a></div><div class="lev2 toc-item"><a href="#Combinar-sobre-índices" data-toc-modified-id="Combinar-sobre-índices-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Combinar sobre índices</a></div><div class="lev2 toc-item"><a href="#Concatenar-en-base-a-un-eje" data-toc-modified-id="Concatenar-en-base-a-un-eje-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Concatenar en base a un eje</a></div><div class="lev2 toc-item"><a href="#Referencias" data-toc-modified-id="Referencias-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Referencias</a></div>

# Combinar DataFrames

Cuando la información está distribuía en varios dataframes, antes de proceder al análisis es necesario combinarlos para crear nuevos dataframes con toda la información necesaria. Existen varias herramientas en Pandas para combinar DataFrames. Veamos algunas de ellas junto con algunos ejemplos ilustrativos de su uso.

In [1]:
import pandas as pd

## Combinar tablas al estilo del álgebra relacional

La función `pd.merge` permite combinar las filas de dos dataframes en función del valor de una o varias columnas. Se trata de la operación *join* al estilo del álgebra relacional. Las columnas de combinación se indican mediante una lista en el argumento `on` de la función. El argumento `how` de la función `pd.merge` permite indicar el tipo de combinación que queremos hacer (`left`, `right`, `outer` o `inner`). El resultado es un nuevo dataframe.

Supongamos que por un lado tenemos información de la cantidad de producción de ciertos productos en distintos paises européos, y por otro lado la información de esos productos. Los datos se muestran en dos dataframes `t1` y `t2`. 

In [2]:
d1 ={'País' : ['Estonia','Estonia', 'Ireland', 'Spain'],
     'Cantidad' : [  8.4,  6.7 ,    227,  58.9 ] , 
     'Producto' : ['A', 'B', 'A', 'Z'] }
     
d2 ={'Producto' : ['A', 'B', 'C'],
     'Descripción' : [  'Leche',  'Cereales',   'Aceite' ] }
    

In [3]:
t1 = pd.DataFrame( d1 )
t1

,Cantidad,País,Producto
0,8.4,Estonia,A
1,6.7,Estonia,B
2,227.0,Ireland,A
3,58.9,Spain,Z


In [4]:
t2 = pd.DataFrame( d2 )
t2

,Descripción,Producto
0,Leche,A
1,Cereales,B
2,Aceite,C


Por ejemplo, la función `pd.merge` permite combinar los dataframes `t1` y `t2`  usando como campos de combinación la columna `Producto`. Veamos ejemplos con distintas opciones:

In [5]:
result = pd.merge(t1, t2, on = ['Producto'])               
result

,Cantidad,País,Producto,Descripción
0,8.4,Estonia,A,Leche
1,227.0,Ireland,A,Leche
2,6.7,Estonia,B,Cereales


Por defecto la función `pd.merge` realiza la operación `inner`. La opción `inner` combina los dataframes devolviendo únicamente aquellas filas que tienen valores idénticos en las columnas que se comparan. 

In [6]:
result = pd.merge(t1, t2, on=['Producto'], how = 'inner')               
result

,Cantidad,País,Producto,Descripción
0,8.4,Estonia,A,Leche
1,227.0,Ireland,A,Leche
2,6.7,Estonia,B,Cereales


En el ejemplo anterior, podemos observar que desaparecen las filas donde el valor de la columna `Producto` tiene valores `C` y `Z`. 

La opción `left` combina dos dataframes devolviendo aquellas filas que tienen valores idénticos en las columnas que se comparan para unir ambas tablas, y todas las filas del dataframe de la izquierda, tengan o no correspondencia con las filas del dataframe de la derecha. Las que no tengan correspondencia se rellenan con `NaN`.

In [7]:
result = pd.merge(t1, t2, on=['Producto'], how='left')
result

,Cantidad,País,Producto,Descripción
0,8.4,Estonia,A,Leche
1,6.7,Estonia,B,Cereales
2,227.0,Ireland,A,Leche
3,58.9,Spain,Z,NaN


En este caso, no existe el valor `Z` en la columna `Producto` del dataframe de la derecha, por lo que el valor de la columna `Descripción` pasa a ser `NaN`. 

Análogamente, la opción `right` combina dos dataframes devolviendo como resultado aquellas filas que tienen valores idénticos en las columnas que se comparan para unir ambas datframes, y todas las filas del dataframe de la derecha, tengan o no correspondencia con las filas del dataframe de la izquierda. Las que no tengan correspondencia se rellenan con `NaN`.

In [8]:
result = pd.merge(t1, t2, how='right', on=['Producto'])
result

,Cantidad,País,Producto,Descripción
0,8.4,Estonia,A,Leche
1,227.0,Ireland,A,Leche
2,6.7,Estonia,B,Cereales
3,NaN,NaN,C,Aceite


La opción `outer` combina los dos datframes devolviendo la unión de las filas devueltas por la opción `left` y `right`:

In [9]:
result = pd.merge(t1, t2,  how='outer', on=['Producto'])
result

,Cantidad,País,Producto,Descripción
0,8.4,Estonia,A,Leche
1,227.0,Ireland,A,Leche
2,6.7,Estonia,B,Cereales
3,58.9,Spain,Z,NaN
4,NaN,NaN,C,Aceite


La función `pd.merge` es muy rica en funcionalidad y los resultados dependen del valor de sus argumentos. Con lo visto anteriormente, podríamos suponer que para poder combinar dos dataframes es necesario que las columnas de combinación se llamen igual. Veamos que esto no es así. 

In [10]:
d3 ={'Id.Producto' : ['A', 'B', 'C'],
     'Descripción' : [  'Leche',  'Cereales',   'Aceite' ] }
t3 = pd.DataFrame( d3 )
t3

,Descripción,Id.Producto
0,Leche,A
1,Cereales,B
2,Aceite,C


Usamos los argumentos `left_on` y `right_on` en lugar de `on` para indicar las columnas de combinación de los dataframes de la izquierda y derecha respectivamente:

In [11]:
result = pd.merge(t1, t3,  left_on = 'Producto', right_on = 'Id.Producto')
result

,Cantidad,País,Producto,Descripción,Id.Producto
0,8.4,Estonia,A,Leche,A
1,227.0,Ireland,A,Leche,A
2,6.7,Estonia,B,Cereales,B


## Combinar sobre índices

En algunos casos, el valor de comparación no se encuentra en las columnas, sino en el índice del dataframe. 

Supongamos que tenemos los dataframes `t4` y `t5`:

In [12]:
t4 = pd.DataFrame( data = [ (92, 89), 
                                (230, 231),
                                (201 , 193),
                                (144, 137)                                ],
                       columns =  [ 'Densidad 2005', 'Densidad 2015'], 
                       index = [ 'ES', 'AL', 'IT', 'CH'])
t4

,Densidad 2005,Densidad 2015
ES,92,89
AL,230,231
IT,201,193
CH,144,137


In [13]:
t5 = pd.DataFrame( data = [ ('España', 46449), 
                                ('Alemania',81197),
                                          ('Japón',127120) ],
                       columns =  ['País', 'Pob 2015'], 
                       index = [ 'ES', 'AL', 'JP'])
t5

,País,Pob 2015
ES,España,46449
AL,Alemania,81197
JP,Japón,127120


Si queremos combinar ambos dataframes usando el valor del índice de ambos, usaremos los argumentos `left_index` y `right_index` como se muestra a continuación:

In [14]:
result = pd.merge(t4, t5, how = 'left', left_index = True, right_index = True)
result

,Densidad 2005,Densidad 2015,País,Pob 2015
ES,92,89,España,46449.0
AL,230,231,Alemania,81197.0
IT,201,193,NaN,NaN
CH,144,137,NaN,NaN


También podemos combinar los dataframes indicando una columna de combinación de un dataframe  y el índice del otro dataframe:

In [15]:
t6 = pd.DataFrame( data = [ ('España', 'ES', 46449), 
                            ('Alemania', 'AL' , 81197),
                            ('Japón', 'JP', 127120) ],
                       columns =  ['País', 'Código', 'Pob 2015'])
t6

,País,Código,Pob 2015
0,España,ES,46449
1,Alemania,AL,81197
2,Japón,JP,127120


In [16]:
result = pd.merge(t6, t4, how = 'left', left_on = 'Código',  right_index = True)
result

,País,Código,Pob 2015,Densidad 2005,Densidad 2015
0,España,ES,46449,92.0,89.0
1,Alemania,AL,81197,230.0,231.0
2,Japón,JP,127120,NaN,NaN


Cuando la combinación de dataframes se realiza en base a los índices, resulta más apropiado utilizar el método `join` de la clase DataFrame. El único requisito para poder utilizar esta operación es que los dataframes involucrados no tengan columnas con el mismo nombre.

In [17]:
t4.join(t5)

,Densidad 2005,Densidad 2015,País,Pob 2015
ES,92,89,España,46449.0
AL,230,231,Alemania,81197.0
IT,201,193,NaN,NaN
CH,144,137,NaN,NaN


## Concatenar en base a un eje

La función `pd.concat` de Pandas permite construir nuevos objetos de tipo `Series` o  `Dataframe` en base a uno de los ejes. Ya vimos las funciones `hstack` y `vstack` definidas en la librería Numpy para apilar arrays horizontal y verticalmente respectivamente. En el caso de Pandas es un poco más complicado, ya que no podemos olvidar que tenemos etiquetas, tanto para las filas como para las columnas.

Veamos algunos ejemplos que muestren el comportamiento de la función `pd.concat`,  tanto para series como para dataframes. Supongamos que tenemos dos series `s1`y `s2`:

In [18]:
s1 = pd.Series( [89, 231, 336],                     
                index = [ 'ES', 'AL', 'JP'])
s1

ES     89
AL    231
JP    336
dtype: int64

In [19]:
s2 = pd.Series( [100, 201 , 144],
                    index = [ 'ES', 'IT', 'CH'])
s2

ES    100
IT    201
CH    144
dtype: int64

En este caso, podemos usar la función `pd.concat` para apilar verticalmente tanto los índices como los valores. Para ello usaremos el argumento `axis` con valor 0:

In [20]:
result = pd.concat([s1, s2], axis = 0)               
result

ES     89
AL    231
JP    336
ES    100
IT    201
CH    144
dtype: int64

También podemos apilar horizontalmente usando el argumento `axis` con valor 1:

In [21]:
result = pd.concat([s1, s2], axis = 1)               
result

,0,1
AL,231.0,NaN
CH,NaN,144.0
ES,89.0,100.0
IT,NaN,201.0
JP,336.0,NaN


El siguiente código permite apilar horizontalmente añadiendo el nombre a las columnas. Para ello hemos usado el argumento `keys`:

In [22]:
result = pd.concat([s1, s2], axis = 1, keys = ['Col 1', 'Col 2'])               
result

,Col 1,Col 2
AL,231.0,NaN
CH,NaN,144.0
ES,89.0,100.0
IT,NaN,201.0
JP,336.0,NaN


Para el caso de objetos de tipo `DataFrame` la mecánica es similar. En el siguiente ejemplo creamos dos dataframes `df1` y  `df2` :

In [23]:
d1 ={'País' : ['Estonia', 'Irlanda', 'Epaña'],
     'Cantidad' : [  8.4,   227,  58.9 ] }
     
d2 ={'Producto' : ['A', 'B'],
     'Descripción' : [  'Leche',  'Cereales',  ] }

df1 = pd.DataFrame(d1, index = ['ES', 'IR', 'SP']) 
df2 = pd.DataFrame(d2, index = ['ES', 'IR']) 

In [24]:
df1

,Cantidad,País
ES,8.4,Estonia
IR,227.0,Irlanda
SP,58.9,Epaña


In [25]:
df2

,Descripción,Producto
ES,Leche,A
IR,Cereales,B


Para apilar ambos dataframes horizontamente, escribimos:

In [26]:
result = pd.concat([df1, df2] , axis = 1)
result

,Cantidad,País,Descripción,Producto
ES,8.4,Estonia,Leche,A
IR,227.0,Irlanda,Cereales,B
SP,58.9,Epaña,NaN,NaN


Para apilar verticalmente, escribimos:

In [27]:
result = pd.concat([df1, df2] , axis = 0)
result

,Cantidad,Descripción,País,Producto
ES,8.4,NaN,Estonia,NaN
IR,227.0,NaN,Irlanda,NaN
SP,58.9,NaN,Epaña,NaN
ES,NaN,Leche,NaN,A
IR,NaN,Cereales,NaN,B


El argumento `ignore_index` permite apilar los dataframes ignorando el valor de los índices:

In [28]:
result = pd.concat([df1, df2] , axis = 0, ignore_index = True)
result

,Cantidad,Descripción,País,Producto
0,8.4,NaN,Estonia,NaN
1,227.0,NaN,Irlanda,NaN
2,58.9,NaN,Epaña,NaN
3,NaN,Leche,NaN,A
4,NaN,Cereales,NaN,B


## Referencias

* [Python Data Analysis Library](http://pandas.pydata.org/)
* [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do)


--------

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />